<a href="https://colab.research.google.com/github/venkateshakula1729/CS253-Python-Assignment/blob/main/Education_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# load data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Feature selection
train_features = train_data[[ "state",  "Party", "Criminal Case","Total Assets", "Liabilities"]]
train_target = train_data["Education"]


# Preprocess numeric features
def convert_numeric_feature(x):
    num_str = x.split()[0]
    num_str = num_str.replace(",", "")
    if "Crore" in x:
      return float(num_str) * 10000000
    elif "Lac" in x:
      return float(num_str) * 100000
    else:
      return float(num_str)

for col in ["Total Assets", "Liabilities"]:
  train_features.loc[:, col] = train_features[col].apply(convert_numeric_feature)
  test_data.loc[:, col] = test_data[col].apply(convert_numeric_feature)

# Encode categorical features using LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
le2 = LabelEncoder()
train_target = le.fit_transform(train_target)
train_features.loc[:, 'Party'] = le1.fit_transform(train_data['Party'])
train_features.loc[:, 'state'] = le2.fit_transform(train_data['state'])

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(train_features, train_target, test_size=0.2, random_state=42)


test_features = test_data[[  "state", "Party",  "Criminal Case", "Total Assets", "Liabilities"]]
test_features.loc[:, 'Party'] = le1.transform(test_data['Party'])
test_features.loc[:, 'state'] = le2.transform(test_data['state'])

########################     KNN        ##############################

Knn_clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')  # n_neighbors = number of neighbors
Knn_clf.fit(X_train, y_train)

y_pred_knn = Knn_clf.predict(X_val)
f1 = f1_score(y_val, y_pred_knn, average='weighted')
print(f"F1-score on validation set (KNN): {f1}")

################################       Dtrees       #######################

# Create a decision tree classifier
dt_clf = DecisionTreeClassifier(max_depth=8, min_samples_split=7, random_state=42)
dt_clf.fit(X_train, y_train)

# Predict on validation set
y_pred_dt = dt_clf.predict(X_val)
f1_dt = f1_score(y_val, y_pred_dt, average='weighted')
print(f"F1-score on validation set (Decision Tree): {f1_dt}")

############################      Randforest        ##########################


# We define a list of values to try for each hyperparameter
# n_estimators_list = range(100,200)
# random_state_list = range(40,50)
# min_samples_split = 7

 # Best model and F1-score trackers
# best_f1 = 0.0
# best_model = None

# for n_estimators in n_estimators_list:
#   for random_state in random_state_list:
#     # Create a Random Forest Classifier object with current parameters
#     rf_clf = RandomForestClassifier(n_estimators=n_estimators, min_samples_split=min_samples_split, random_state=random_state)

#     # Train the model
#     rf_clf.fit(X_train, y_train)

#     # Predict on validation set and calculate F1-score
#     y_pred_rf = rf_clf.predict(X_val)
#     f1_score_current = f1_score(y_val, y_pred_rf, average='weighted')

#     if f1_score_current > best_f1:
#       best_f1 = f1_score_current
#       best_model = rf_clf

# # Print the best hyperparameters and F1-score found
# print(f"Best Hyperparameters: n_estimators={best_model.n_estimators}, min_samples_leaf={min_samples_leaf}, random_state={best_model.random_state}")
# print(f"Best F1-score on Validation set: {best_f1}")

rf_clf = RandomForestClassifier(n_estimators=300, random_state=2, min_samples_split=7,  bootstrap=False)
rf_clf.fit(X_train, y_train)

# Predict on validation set
y_pred_rf = rf_clf.predict(X_val)
f1_rf = f1_score(y_val, y_pred_rf, average='weighted')
print(f"F1-score on validation set (Random Forest): {f1_rf}")

test_pred_rf = rf_clf.predict(test_features)
decoded_predictions = le.inverse_transform(test_pred_rf)

submission = pd.DataFrame({"ID": test_data["ID"], "Education": decoded_predictions})
submission.to_csv("submission.csv", index=False)
